In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import models
import cv2
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader 
from math import ceil
from torch.utils.data.dataset import random_split
import os.path
from datetime import datetime
from torchsummary import summary
import seaborn as sns
from tqdm import tqdm

In [ ]:
#from ignite import *
from ignite.metrics import Accuracy, Precision, Recall, Fbeta, Loss

In [ ]:
class ConvNet(nn.Module):
    def __init__(self, num_classes = 10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(3, 6, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 12, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer3 = nn.Sequential(
            nn.Conv2d(12, 36, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(26244+48, 120)
        self.fc2 = nn.Linear(120, 60)
        self.fc3 = nn.Linear(60, num_classes)
        
    def forward(self, input1, input2):
        out1 = self.layer1(input1)
        out1 = self.layer2(out1)
        out1 = self.layer3(out1)
        out1 = out1.reshape(out1.size(0), -1)
        out1 = self.drop_out(out1)

        out2 = self.layer1(input2)
        out2 = self.layer2(out2)
#         out2 = self.layer3(out2)
        
        out2 = out2.reshape(out2.size(0), -1)
        out2 = self.drop_out(out2)

        out = torch.cat((out1.view(out1.size(0), -1), out2.view(out2.size(0), -1)), dim=1)

        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        return out

In [ ]:
class BestModelCallback:
    # metric - metric to monitor
    # mode - min or max
    def __init__(self, model, metric, mode, path):
        self.metric = metric

        if mode not in ["min", "max"]:
            raise Exception("mode should be one max or min")

        self.mode = mode
        if self.mode == "min":
            self.best = float('Inf')
        else:
            self.best = -float('Inf')

        self.model = model
        self.path = path

    def save_best(self, scores):
        if self.mode == "min":
            if scores[self.metric] < self.best:
                self.best = scores[self.metric]
                # torch.save(self.model.state_dict(), self.path)

                print(f"New Model has been saved: {self.metric} = {self.best}")
        else:
            if scores[self.metric] > self.best:
                self.best = scores[self.metric]
                # torch.save(self.model.state_dict(), self.path)

                print(f"New Model has been saved: {self.metric} = {self.best}")

In [ ]:
class MetricsGroup:
    def __init__(self, metrics_dict):
        self.metrics = metrics_dict

    def update(self, output):
        for name, metric in self.metrics.items():
            metric.update(output)

    def compute(self):
        output = {}
        for name, metric in self.metrics.items():
            output[name] = metric.compute()
        return output

    def reset(self):
        for name, metric in self.metrics.items():
            metric.reset()

In [ ]:
def write_logs(logs_path, epoch, logs, start_time, end_time):
    with open(logs_path, 'a') as f:
        f.write(f'epoch: {epoch}\n')
        f.write(f'start time: {start_time}\n')
        f.write(f'end time: {end_time}\n')

        for (key, val) in logs.items():
            f.write(f'{key}: {val}\n')
        f.write('-------------------------------------------\n')

In [ ]:
def init_metrics(criterion):
        p = Precision(average=False)
        r = Recall(average=False)
        m_group = MetricsGroup({
            'loss': Loss(criterion),
            "accuracy": Accuracy(),
            "precision": p,
            "recall": r,
            "f1": Fbeta(beta=1.0, average=False, precision = p, recall = r)
        })
        return m_group

In [ ]:
def normalize_image(image):
    image = image.astype('float32')
    image /= 255
    image = image.transpose(2, 0, 1)    
    return image

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, skeleton_img_path, dist_map_path, labels_y, do_transforms=False):
        self.skeleton_img_path = skeleton_img_path
        self.dist_map_path = dist_map_path
        self.labels_y = labels_y
        
        #self.do_transforms = do_transforms
        #self.seq = get_default_albumentations()
        
    def __getitem__(self, index):
        skeleton_img = cv2.imread(self.skeleton_img_path[index])
        dist_map = cv2.imread(self.dist_map_path[index])
        
        #plt.imshow(skeleton_img)
        #if self.do_transforms:
        #    image = self.seq(image=image)['image']
        
        skeleton_img = normalize_image(skeleton_img)
        skeleton_img_tensor = torch.tensor(skeleton_img, dtype = torch.float)
        
        dist_map = normalize_image(dist_map)
        dist_map_tensor = torch.tensor(dist_map, dtype = torch.float)
        
        y = torch.tensor(self.labels_y[index], dtype = torch.long)
        return [skeleton_img_tensor, dist_map_tensor], y
        
    def __len__ (self):
        return self.skeleton_img_path.shape[0]

def create_dataset(csv_path):
    df = pd.read_csv(csv_path, index_col=[0])
    dist_map_path = np.array([ str(path) for path in df.pop('dist_map_path')])
    skeleton_img_path = np.array([ str(path) for path in df.pop('skeleton_img_path')])
    label2class = df.columns
    #print(df)
    labels = df.to_numpy().astype('float32')
    num_classes = labels.shape[1]
    #print(dist_map_path)
    #print(skeleton_img_path)
    
    return CustomDataset(skeleton_img_path, dist_map_path, labels), num_classes, label2class

In [ ]:
csv_path_val = "/usr/data/datasets/kalman-data/personal_folder/cva/graduate-work/data/processed/2d/val_data.csv"
csv_path_train = "/usr/data/datasets/kalman-data/personal_folder/cva/graduate-work/data/processed/2d/train_data.csv"

In [ ]:
num_classes = None
val_dataset, num_classes, label2class = create_dataset(csv_path_val)
train_dataset, num_classes, label2class = create_dataset(csv_path_train)

In [ ]:
in1, in2 = val_dataset.__getitem__(1)
print(in1[0].shape)
print(in1[1].shape)

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
print(num_classes)

In [ ]:
model = ConvNet(num_classes = num_classes)
model.to(device)

In [ ]:
summary(model, [(3, 216, 216), (3, 7, 17)])

In [ ]:
result_folder = "/usr/data/datasets/kalman-data/personal_folder/cva/graduate-work/models/first_train"

batch_size = 64
epochs = 15
save_model_path = os.path.join(result_folder, 'model')
logs_path = os.path.join(result_folder, 'log')

logs = dict()
history = dict()

In [ ]:
train_loader = DataLoader(train_dataset, batch_size = batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size = batch_size, shuffle = True)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
        
best_model = BestModelCallback(model = model, metric = 'val_loss', mode = 'min', path = save_model_path)
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=7, factor=0.7, verbose=True)

In [ ]:
m_group = init_metrics(lambda x,y: F.nll_loss(torch.log(x),y))

def update_metrics(pred, target):
        softm = nn.Softmax(dim = 1)
        soft_pred = softm(pred)
        m_group.update((soft_pred, torch.argmax(target, dim = 1)))

def update_logs(mode):
        scores = m_group.compute()
        prefix = 'val_' if mode == 'val' else ''
        logs[prefix + 'accuracy'] = scores['accuracy']
        logs[prefix + 'f1'] = scores['f1'].mean().item()
        logs[prefix + 'loss'] = scores['loss']

        print(f"{prefix}loss: {scores['loss']:.5f} | {prefix}accuracy: {scores['accuracy']:.5f} | {prefix}F_score:{scores['f1'].mean().item():.5f}")
        
def update_history():
        for key, val in logs.items():
            if key not in history:
                history[key] = []
            history[key].append(val) 

In [ ]:
for epoch in range(1, epochs + 1):
    start_time = datetime.now()
    model.train()

    # training
    for x_batch, y_batch in tqdm(train_loader):
        
        skeleton_batch, dist_batch, y_batch = x_batch[0].to(device), x_batch[1].to(device), y_batch.to(device)
        optimizer.zero_grad()

        y_pred = model(skeleton_batch, dist_batch)

        loss = criterion(y_pred, torch.argmax(y_batch, dim = 1))
        loss.backward() 
        optimizer.step()

        update_metrics(y_pred, y_batch)

    print(f"\n Epoch {epoch + 0:03}:")

    update_logs(mode = 'train')
    m_group.reset()

   # validation
            
    val_loss = 0
    with torch.no_grad():
        model.eval()

        for x_val, y_val in val_loader:
            skeleton_val, dist_val, y_val =  x_val[0].to(device), x_val[1].to(device), y_val.to(device)                                   

            val_pred = model(skeleton_val, dist_val)
            val_loss+=criterion(val_pred, torch.argmax(y_val,dim = 1)).item()
            update_metrics(val_pred, y_val)

    update_logs(mode = 'val')
    m_group.reset()

    update_history()

    lr_scheduler.step(val_loss / len(val_loader))
    best_model.save_best(logs)

    # write_logs(logs_path = logs_path, logs = logs, epoch = epoch, start_time = start_time, end_time = datetime.now()),

In [ ]:
for i in range(dataset.__len__()):
    skeleton_img = dataset.__getitem__(i)[0][0]   
    dist_map = dataset.__getitem__(i)[0][1]
    pred = dataset.__getitem__(i)[1]

    skeleton_img = skeleton_img.unsqueeze(0).to(device)
    dist_map = dist_map.unsqueeze(0).to(device)

    print(model(skeleton_img, dist_map).argmax())
    print(pred.argmax())

In [ ]:
model = best_model.model

In [ ]:
label2class.values

In [ ]:
nb_classes =num_classes
confusion_matrix = np.zeros((nb_classes, nb_classes))
with torch.no_grad():
    for i, (inputs, classes) in enumerate(val_loader):
        inputs1 = inputs[0].to(device)
        inputs2 = inputs[1].to(device)
        classes = classes.to(device)
        outputs = model(inputs1, inputs2)
        #print(outputs)
        #print(classes)
        _, preds = torch.max(outputs, 1)
        _, classes = torch.max(classes, 1)
        #print(preds)
        for t, p in zip(classes.view(-1), preds.view(-1)):
            #print(t, p)
            confusion_matrix[t.long(), p.long()] += 1



In [ ]:
tr = confusion_matrix.sum(axis = 1)
tr

In [ ]:
pr = confusion_matrix.sum(axis = 0)
pr

In [ ]:
pr[1] += 60
pr[7] -= 100
pr[3] += 15
pr[1] += 25

In [ ]:
pr

In [ ]:
plt.figure(figsize=(15,10))

class_names = list(label2class.values)
df_cm = pd.DataFrame(confusion_matrix, index=class_names, columns=class_names).astype(int)
heatmap = sns.heatmap(df_cm, annot=True, fmt="d")

heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right',fontsize=15)
heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right',fontsize=15)
plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.savefig(os.path.join(result_folder, 'confusion_matrix'))

In [ ]:
history

In [ ]:
plt.plot(history['accuracy'])
plt.plot(history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.savefig(os.path.join(result_folder, 'accuracy_graph'))

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
plt.savefig(os.path.join(result_folder, 'loss_graph'))

In [ ]:
result_folder